In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df_gens_anti_R =pd.read_csv('df_gens_anti_R.csv')
final_table_df = pd.read_csv('Final_Table.csv')
antibiotic_family = pd.read_csv('antibiotic_family.csv')

### preprocessing df_gens_anti_R 

In [ ]:
df_gens_anti_R.shape

In [ ]:
grouped_df_gens_anti_R = df_gens_anti_R.groupby(['aro_term', 'drug_class', 'amr_gene_family']).first().reset_index()
grouped_df_gens_anti_R.shape

In [ ]:
grouped_df_gens_anti_R = grouped_df_gens_anti_R.iloc[:, :3]

In [ ]:
grouped_df_gens_anti_R

In [ ]:
import pandas as pd

# Create an empty DataFrame to store the updated rows
updated_rows1 = pd.DataFrame(columns=['aro_term', 'drug_class', 'amr_gene_family'])

# Iterate over the rows of the original DataFrame
for index, row in grouped_df_gens_anti_R.iterrows():
    drug_classes = row['drug_class'].split(';')  # Split the 'drug_class' values by ';'

    # Create new rows for each 'aro_term' and separated 'drug_class'
    for drug_class in drug_classes:
        if drug_class.endswith(" antibiotic"):
            drug_class = drug_class[:-11]  # Remove the last 10 characters (" antibiotic")
        if drug_class.startswith(" "):
            drug_class = drug_class[1:]  # Remove leading space
        updated_rows1 = pd.concat([updated_rows1, pd.DataFrame({'aro_term': [row['aro_term']], 'drug_class': [drug_class], 'amr_gene_family': [row['amr_gene_family']]})])

# Print the updated DataFrame
print(updated_rows1)


In [ ]:
grouped_df_gens_anti_R = updated_rows1
grouped_df_gens_anti_R

In [ ]:
grouped_df_gens_anti_R['amr_gene_family_transformed'] = np.where(
    grouped_df_gens_anti_R['amr_gene_family'].str.contains('beta-lactamase'),
    'beta-lactamase',
    np.where(
        grouped_df_gens_anti_R['amr_gene_family'].str.contains('quinolone'),
        'quinolone',
        grouped_df_gens_anti_R['amr_gene_family']
    )
)

In [ ]:
grouped_df_gens_anti_R['amr_gene_family'] = grouped_df_gens_anti_R['amr_gene_family_transformed']
grouped_df_gens_anti_R.iloc[:, :3]

In [ ]:
final_df = final_table_df.copy()
final_df['R'] = None
final_df = final_df.rename(columns={'label': 'S'})
try:
    for index, row in grouped_df_gens_anti_R.iterrows():
        gene = str(row['aro_term']).lower()
        drug_class = str(row['drug_class']).lower()
        amr_gene_family = str(row['amr_gene_family']).lower()
        mask = (final_df['gene'].str.lower() == gene) & ((final_df['drug_class'].str.lower() == drug_class) | (final_df['drug_class'].str.lower() == amr_gene_family))
        final_df.loc[mask, 'R'] = 1
except Exception as e:
    print('An error occurred:', str(e))
    final_df.to_csv("final_df_lower_exception_20230530_A.csv", index=False)

final_df.to_csv("final_df_lower_success_20230530_A.csv", index=False)


In [ ]:
df_old = pd.read_csv(r'C:\Users\taliya shitreet\PycharmProjects\NER\final_df_lower_success.csv')
df_old

In [75]:
final_df['S'] = final_df['S'].replace('S', 1)
count_R_old = df_old['label'].value_counts()['R']
count_R_new = final_df['R'].value_counts()[1]
count_S_old = df_old['label'].value_counts()['S']
count_S_new = final_df['S'].value_counts()[1]
# Print the count
print("count_R_old: ",count_R_old, "\ncount_R_new: ", count_R_new, "\ncount_S_old: ",count_S_old, "\ncount_S_new: ", count_S_new)

count_R_old:  10319 
count_R_new:  38985 
count_S_old:  14235 
count_S_new:  14235


In [76]:
count_null_rows = len(final_df[(final_df['R'].isnull()) & (final_df['S'].isnull())])
# Print the count
print(count_null_rows)

211494


In [77]:
count_not_null_rows = len(final_df[(final_df['R'].notnull()) | (final_df['S'].notnull())])
# Print the count
print(count_not_null_rows)

52890
